## Neo4j Import of GraphRAG Result Parquet files
This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization.

You can create a free Neo4j Graph database (free AuraDB) [here](https://neo4j.com/product/auradb/).

### How does it work?
The notebook loads the parquet files from the output folder of your indexing process and loads them into Pandas dataframes. It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships.

All operations use `MERGE`, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement
```
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [1]:
GRAPHRAG_FOLDER="artifacts"

In [2]:
import pandas as pd
from neo4j import GraphDatabase
import time

In [ ]:
NEO4J_URI="neo4j+s://name.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="password"
NEO4J_DATABASE="neo4j"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver

In [4]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, 
    and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0, total, batch_size):
        batch = df.iloc[start: min(start+batch_size, total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in {time.time() - start_s} s.')    
    return total

### Indexes and Constraints
Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect. Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [5]:
# create constraints, idempotent operation

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process
### Importing the Documents
We're loading the parquet file for the documents and create nodes with their ids and add the title property. We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [6]:
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/documents.parquet', 
                        columns=["id", "human_readable_id", "title", "creation_date", "metadata"])
doc_df.head(2)

,id,human_readable_id,title,creation_date,metadata
0,6852a66a08b38d4d412f043ca78ada50c66b2bb5e4724a...,1,penitencia.txt,2025-03-26 21:49:16 +0100,None


In [7]:
# import documents
document_statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title, .human_readable_id, .creation_date, .metadata}
"""
batched_import(document_statement, doc_df)

{'_contains_updates': True, 'labels_added': 1, 'nodes_created': 1, 'properties_set': 4}
1 rows in 0.1596050262451172 s.


1

### Loading Text Units
We load the text units, create a node per id and set the text and number of tokens. Then we connect them to the documents that we created before.

In [8]:
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/text_units.parquet',
                        columns=["id", "text", "n_tokens", "document_ids", 
                                "entity_ids", "relationship_ids", "covariate_ids"])
text_df.head(2)

,id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,b65bb9f86e8bf83b6a5c7e814c2bfb7d0d65bc4135b17f...,Cap&#237;tulo 1 1 Llevaba recorridos poco más ...,1200,[6852a66a08b38d4d412f043ca78ada50c66b2bb5e4724...,"[62b6d127-5d34-4e9e-aaa2-b3ca31dcfe8a, bb88600...","[8376b416-8e30-40aa-883d-d244e3271841, b6d9666...",[]
1,dc1d18ef85ca582714aa0a57b840c790e7eb76a5fba54e...,su mensaje tal y como lo habían diseñado. Jon...,1200,[6852a66a08b38d4d412f043ca78ada50c66b2bb5e4724...,"[bb886006-70c3-4aec-9693-9f34e1e0fb40, 750da1e...","[9478efec-44fb-45a9-b158-3812d39d6532, a231d3e...",[]


In [9]:
text_unit_statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens},
    c.name = 'Chunk',
    c.entity_ids = value.entity_ids,
    c.relationship_ids = value.relationship_ids,
    c.covariate_ids = value.covariate_ids
WITH c, value
UNWIND value.document_ids AS document_id
MATCH (d:__Document__ {id:document_id})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(text_unit_statement, text_df)

{'_contains_updates': True, 'labels_added': 109, 'relationships_created': 109, 'nodes_created': 109, 'properties_set': 763}
109 rows in 0.6662468910217285 s.


109

### Loading Nodes
For the nodes we store id, name, description, embedding (if available), human readable id.

In [10]:
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/entities.parquet',
                          columns=["id", "human_readable_id", "title", "type", "description", 
                                  "text_unit_ids", "frequency", "degree", "x", "y"])
entity_df.head(2)

,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,62b6d127-5d34-4e9e-aaa2-b3ca31dcfe8a,0,MADRID,GEO,"Madrid is the capital city of Spain, known for...",[b65bb9f86e8bf83b6a5c7e814c2bfb7d0d65bc4135b17...,6,4,0.0,0.0
1,bb886006-70c3-4aec-9693-9f34e1e0fb40,1,JON,PERSON,Jon is a complex character navigating a tumult...,[b65bb9f86e8bf83b6a5c7e814c2bfb7d0d65bc4135b17...,82,218,0.0,0.0


In [11]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e.name = coalesce(value.title, value.id),
    e += value {.human_readable_id, .title, .description, .frequency, .degree, .x, .y},
    e.text_unit_ids = value.text_unit_ids
WITH e, value
CALL apoc.create.addLabels(e, 
      case when coalesce(value.type,"") = "" then [] 
      else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) 
YIELD node
RETURN count(*)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 392, 'nodes_created': 392, 'properties_set': 3920}
392 rows in 0.6980679035186768 s.


392

### Import Relationships
For the relationships we find the source and target node by name, using the base `__Entity__` type. After creating the RELATED relationships, we set the description as attribute.

In [12]:
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/relationships.parquet',
                       columns=["id", "source", "target", "description", "weight", 
                               "combined_degree", "text_unit_ids"])
rel_df.head(2)

,id,source,target,description,weight,combined_degree,text_unit_ids
0,8376b416-8e30-40aa-883d-d244e3271841,MADRID,JON,"Jon is leaving Madrid, indicating a significan...",8.0,222,[b65bb9f86e8bf83b6a5c7e814c2bfb7d0d65bc4135b17...
1,b6d96662-eaa9-45e5-8e13-6da4a01071d5,JON,COCHE DEPORTIVO,"Jon had to leave his sports car behind, which ...",7.0,219,[b65bb9f86e8bf83b6a5c7e814c2bfb7d0d65bc4135b17...


In [13]:
rel_statement = """
// Match source and target by ID or name, prioritizing exact ID match
OPTIONAL MATCH (source:__Entity__ {id: value.source}) 
WITH source, value
OPTIONAL MATCH (source2:__Entity__ {name: value.source}) 
WITH coalesce(source, source2) as source, value
WHERE source IS NOT NULL

OPTIONAL MATCH (target:__Entity__ {id: value.target})
WITH source, target, value
OPTIONAL MATCH (target2:__Entity__ {name: value.target})
WITH source, coalesce(target, target2) as target, value
WHERE target IS NOT NULL

// Create relationship and set properties
MERGE (source)-[rel:RELATED {id: value.id}]->(target)
SET rel += value {.weight, .description, .combined_degree}
SET rel.text_unit_ids = value.text_unit_ids
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 496, 'properties_set': 2480}
496 rows in 1.8233850002288818 s.


496

### Importing Communities
For communities we import their id, title, level. We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [14]:
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/communities.parquet', 
                             columns=["id", "community", "level", "parent", "children",
                                     "title", "entity_ids", "relationship_ids", 
                                     "text_unit_ids", "period", "size"])

community_df.head(2)

,id,community,level,parent,children,title,entity_ids,relationship_ids,text_unit_ids,period,size
0,49481fe5-d264-48b4-aadf-825ffadfa4dc,0,0,-1,"[12, 13]",Community 0,"[bb886006-70c3-4aec-9693-9f34e1e0fb40, 0397ff7...","[002dd161-6def-4b65-b5d8-9e724d1dbc06, 00d323b...",[04f60d35ff6ea1d13757a9caa8f310a5ff40558dc46a4...,2025-03-28,131
1,5cfb4792-a54c-4dbf-a88a-a7a569a2f102,1,0,-1,"[14, 15, 16]",Community 1,"[fb50f1da-c96e-4a84-9cf1-084665ff6b45, 6d0fee6...","[0179caa7-01ac-4af2-9557-749d52f78031, 0803512...",[0432d276c182aa29456c067d9ae892765781722497b33...,2025-03-28,66


In [15]:
community_statement = """
MERGE (c:__Community__ {id:value.id})
SET c.community = coalesce(value.community, value.id),
    c += value {.level, .title, .parent, .children, .period, .size},
    c.entity_ids = value.entity_ids,
    c.relationship_ids = value.relationship_ids,
    c.text_unit_ids = value.text_unit_ids
WITH c, value
UNWIND value.entity_ids as entity_id
MATCH (e:__Entity__ {id:entity_id})
MERGE (e)-[:IN_COMMUNITY]->(c)
WITH c, value
UNWIND value.relationship_ids as rel_id
MATCH ()-[r:RELATED {id:rel_id}]->()
SET r.community_ids = CASE 
  WHEN r.community_ids IS NULL THEN [c.id] 
  WHEN NOT c.id IN r.community_ids THEN r.community_ids + c.id 
  ELSE r.community_ids
END
"""

batched_import(community_statement, community_df)

{'_contains_updates': True, 'labels_added': 41, 'relationships_created': 681, 'nodes_created': 41, 'properties_set': 47698}
41 rows in 4.619127035140991 s.


41

### Importing Community Reports
Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about. For the findings we create the findings in context of the communities.

In [16]:
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/community_reports.parquet',
                                    columns=["id", "human_readable_id", "community", "level", 
                                            "parent", "children", "title", "summary", 
                                            "full_content", "rank", "rating_explanation", 
                                            "findings", "period", "size"])
community_report_df.head(2)

,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,period,size
0,1603b040040d4ab2966d690bf7bf47fa,12,12,1,0,[],Jon's Emotional Landscape and Public Persona,"The community centers around Jon, a complex ch...",# Jon's Emotional Landscape and Public Persona...,7.5,The impact severity rating is high due to Jon'...,[{'explanation': 'Jon is depicted as an actor ...,2025-03-28,129
1,f05099adb1ed4603832f1957880c415a,13,13,1,0,[],Village and Night: A Tension-Filled Community,The community is centered around the Village a...,# Village and Night: A Tension-Filled Communit...,6.5,The impact severity rating is moderate to high...,[{'explanation': 'The Village serves as a sign...,2025-03-28,2


In [17]:
# import communities
community_report_statement = """
MATCH (c:__Community__ {community:value.community})
SET c += value {.human_readable_id, .level, .title, .parent, .children, 
                .rank, .rating_explanation, .full_content, .summary,
                .period, .size}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id: c.id + '-' + finding_idx})
SET f += finding
"""

batched_import(community_report_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 186, 'relationships_created': 186, 'nodes_created': 186, 'properties_set': 1009}
41 rows in 0.7577090263366699 s.


41

### Importing Covariates
Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

**By default, covariates are not included in the output, so the file might not exists in your output if you didn't set the configuration to extract claims**

In [ ]:
cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/covariates.parquet')
cov_df.head(2)

In [ ]:
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
batched_import(cov_statement, cov_df)